In [1]:
# To run this notebook create a new conda environment with:
# conda create -n coda_env python=3.6 scipy numpy scikit-bio

import numpy as np
import scipy.stats
from skbio.stats.composition import *
from scipy.optimize import minimize

$n$ = number of samples

$d$ = number of parts in composition

$k$ = number of independent variables (compositions) in regression model

# Regression Model

Let $V, U^{(1)}, \ldots, U^{(k)}$ be in $S^d_n.$

The regression model is given by

$$ \hat{V} = \beta_1 \otimes U^{(1)} \oplus \ldots \oplus \beta_k \otimes U^{(k)}$$

After ilr-transform, this becomes

$$ \hat{Y} = \beta_1 \times X^{(1)} + \ldots + \beta_k \times X^{(k)}$$

Note: Since there is no obvious way to include a constant term in this model, H. Wang et al first centralizes the observed data, and un-centers the predicted values to give the final fitted values. 

The $\beta = (\beta_1,\ldots, b_k)$ which minimizes the sum of squared errors between $Y, \hat{Y}$ is given in eq (58) (H. Wang et al, 2013).

Similarly to eq 9 of (M. Avalos et al, Representation learning of Composition Data, NeurIPS 2018), we can instead consider the gauged-KL loss 

$$ \ell = D_{\exp} \left( \left( \sum_k \beta_k X^{(k)} \right) W, clr(V) \right) $$

$$ = D_{KL} \left( \tilde{V}, \exp\left( \left( \sum_k \beta_k X^{(k)} \right) W \right) \right) $$

$$ = ( \mathbf{1}_{n\times 1} )^T \exp\left( \left( \sum_k \beta_k X^{(k)} \right) W \right) \mathbf{1}_{d\times 1} - \operatorname{trace}\left( \tilde{V}^T \left( \sum_k \beta_k X^{(k)} \right) W \right) + const$$

Gradient: Let $g$ and $h$ denote the first and second term in the gauged-KL-loss respectively.

$$ g(\beta) = \sum_{i=1}^d \sum_{j=1}^n \exp \left( \beta_1 C^{(1)}_{ij} + \ldots + \beta_k C^{(k)}_{ij} \right)$$
$$ h(\beta) = \operatorname{trace}\left( \tilde{V}^T \left( \sum_k \beta_k C^{(k)} \right)  \right) = \sum_k \beta_k \operatorname{trace}\left( \tilde{V}^T C^{(k)} \right) $$
We have $$ \frac{ \partial g}{\partial \beta_r } = \sum_{i=1}^d \sum_{j=1}^n C^{(r)}_{ij} \exp \left( \beta_1 C^{(1)}_{ij} + \ldots + \beta_k C^{(k)}_{ij} \right)$$

$$ \frac{ \partial h}{\partial \beta_r } = \operatorname{trace}\left( \tilde{V}^T C^{(r)}\right)$$

This loss function is convex in $\beta$ and we can compute the gradient, so we can find the optimal $\beta$ by standard methods. 

Observation - Both the sum of squared errors and the gauged-KL loss produce $\beta$ which seem to produce fairly similar results under a variety of evaluation metrics.

In [2]:
def KL_coeff(V, U_vec):
    k = len(U_vec)
    V_tilde = np.exp(clr(V))
    C_vec = [clr(U) for U in U_vec]
    
    def kl_loss(beta):
        # beta in R^k
        ilr_estimator = sum(beta[i] * C_vec[i] for i in range(k))
        h = sum(beta[i] * np.trace(V_tilde.T @ C_vec[i]) for i in range(k))
        return np.sum(np.exp(ilr_estimator)) - h
    
    def kl_loss_grad(beta):
        ilr_estimator = sum(beta[i] * C_vec[i] for i in range(k))
        return np.array([np.sum(C_vec[i] * np.exp(ilr_estimator)) - np.trace(V_tilde.T@C_vec[i]) for i in range(k)])
    
    return minimize(kl_loss, np.array([0.0]*k), method='BFGS', jac=kl_loss_grad, tol=1e-16,
                    options={'gtol': 1e-012, 'disp':False}).x

In [3]:
def L2_coeff(V, U_vec):
    k = len(U_vec)
    Y = ilr(V)
    X_vec = [ilr(U) for U in U_vec]
    A = np.array([ [np.trace(X_vec[i].T @ X_vec[j]) for i in range(k)] for j in range(k)])
    b = np.array( [np.trace(X_vec[i].T@Y) for i in range(k)]).T
    return np.linalg.solve(A,b)

In [4]:
# Calculated directly in evaluate_XX_model functions below.
#def fitted_outputs(beta, U_vec, V_mean):
#    return clr_inv( sum(beta[i]* clr(U_vec[i]) for i in range(len(U_vec))) + clr(V_mean))

In [5]:
def L1_error(V, V_):
    return np.sum(np.absolute(V - V_))

def L2_error(V, V_):
    return np.sum(np.sqrt(np.sum(np.square(V - V_), axis=1)))

def Fisher_error(V, V_):
    return 2*np.trace(np.arccos(np.sqrt(V)@np.sqrt(V_.T)))

def symKL_error(V, V_):
    return np.sum((V-V_) * np.log(np.divide(V, V_)))

def report_evaluation(V_obs, V_fit):
    print("Fisher-Rao:   ", Fisher_error(V_obs, V_fit))
    print("Symmetric KL: ", symKL_error(V_obs, V_fit))
    print("L2 error:     ", L2_error(V_obs, V_fit))
    print("L1 error:     ", L1_error(V_obs, V_fit))

The functions 'eval_XX_model' employs a train/test split as usual and is what we use in evaluating the effectiveness of the models. To reproduce the results in H. Wang (2013) where they both fit and test their model over the entire dataset, we can use 'eval_XX_model(V, V, U_vec, U_vec)'. 

In [6]:
def eval_KL_model(V_train, V_test, U_train_vec, U_test_vec, center_data = True):
    k = len(U_train_vec)
    if center_data:
        V_train_centered = centralize(V_train)
        V_train_mean = scipy.stats.gmean(V_train, axis=0)
        U_train_centered = [centralize(U) for U in U_train_vec]
        U_test_centered = [centralize(U) for U in U_test_vec]
        
        beta_KL = KL_coeff(V_train_centered, U_train_centered)
        V_prediction = clr_inv( sum(beta_KL[i]* clr(U_test_centered[i]) for i in range(k)) 
                               + clr(np.array([V_train_mean]*V_test.shape[0])) )
        report_evaluation(V_test, V_prediction)
    else: # Do not center the data
        beta_KL = KL_coeff(V_train, U_train_vec)
        V_prediction = clr_inv( sum(beta_KL[i]* clr(U_test_vec[i]) for i in range(k)))
        report_evaluation(V_test, V_prediction)

In [7]:
def eval_L2_model(V_train, V_test, U_train_vec, U_test_vec, center_data = True):
    k = len(U_train_vec)
    if center_data:
        V_train_centered = centralize(V_train)
        V_train_mean = scipy.stats.gmean(V_train, axis=0)
        U_train_centered = [centralize(U) for U in U_train_vec]
        U_test_centered = [centralize(U) for U in U_test_vec]
        
        beta_L2 = L2_coeff(V_train_centered, U_train_centered)
        V_prediction = clr_inv( sum(beta_L2[i]* clr(U_test_centered[i]) for i in range(k)) 
                               + clr(np.array([V_train_mean]*V_test.shape[0])) )
        report_evaluation(V_test, V_prediction)
    else: # Do not center the data
        beta_L2 = L2_coeff(V_train, U_train_vec)
        V_prediction = clr_inv( sum(beta_L2[i]* clr(U_test_vec[i]) for i in range(k)))
        report_evaluation(V_test, V_prediction)

# Summary of results below

It generally appears that the L2 model performs slightly better than the KL model when we center-uncenter the data, and that the KL model slightly outperforms the L2 model when we only use the raw data. 

For the real world datasets (the first two), the Centered models greatly outperform the Uncentered models.
Very strangely, for the artifical dataset created below the Uncentered models appears to greatly outperform the Centered models.

# Economic Data from (H. Wang et al, Multiple linear regression for Compositional Data, 2013)

https://www.sciencedirect.com/science/article/pii/S0925231213005808

In [8]:
V = np. array([
       [0.024, 0.568, 0.408],
       [0.023, 0.540, 0.437],
       [0.021, 0.516, 0.463],
       [0.019, 0.493, 0.488],
       [0.018, 0.474, 0.508],
       [0.016, 0.463, 0.521],
       [0.015, 0.461, 0.524],
       [0.014, 0.457, 0.529],
       [0.012, 0.479, 0.509],
       [0.010, 0.482, 0.508],
       [0.010, 0.474, 0.516],
       [0.009, 0.470, 0.521],
       [0.008, 0.446, 0.546],
       [0.008, 0.432, 0.560],
       [0.007, 0.399, 0.594],
       [0.007, 0.420, 0.573]])

U1 = np.array([
       [0.09853, 0.54467, 0.35680],
       [0.12044, 0.52255, 0.35701],
       [0.12708, 0.49105, 0.38187],
       [0.12443, 0.46027, 0.41530],
       [0.11414, 0.46458, 0.42128],
       [0.10772, 0.44310, 0.44918],
       [0.11004, 0.39875, 0.49121],
       [0.10154, 0.39671, 0.50175],
       [0.08626, 0.40765, 0.50609],
       [0.06878, 0.45352, 0.47770],
       [0.06296, 0.42429, 0.51275],
       [0.05504, 0.41676, 0.52820],
       [0.05243, 0.41251, 0.53505],
       [0.04688, 0.40272, 0.55040],
       [0.05119, 0.37391, 0.57490],
       [0.03930, 0.37573, 0.58496]])

U2 = np.array([
       [0.00350, 0.29244, 0.70406],
       [0.00380, 0.30635, 0.68985],
       [0.00192, 0.30276, 0.69531],
       [0.00117, 0.31051, 0.68832],
       [0.00297, 0.30802, 0.68901],
       [0.00308, 0.28961, 0.70732],
       [0.00377, 0.30051, 0.69573],
       [0.00264, 0.30085, 0.69651],
       [0.00192, 0.29305, 0.70503],
       [0.00078, 0.30147, 0.69775],
       [0.00176, 0.25639, 0.74185],
       [0.00420, 0.26713, 0.72867],
       [0.00220, 0.25783, 0.73997],
       [0.00209, 0.24018, 0.75773],
       [0.00235, 0.22705, 0.77060],
       [0.00360, 0.24734, 0.74906]])

In [9]:
# Checking V_mean as stated in H.Wang2013 is correct.
# NOTE - This differs slightly from the value published in their paper:
# [0.01275, 0.47401, 0.51324]
# Potentially an error in their calculations
scipy.stats.gmean(V, axis=0)

array([0.01268806, 0.47162267, 0.51065353])

In [10]:
V_train, V_test = V[::2], V[1::2]
U1_train, U1_test = U1[::2], U1[1::2]
U2_train, U2_test = U2[::2], U2[1::2]

In [11]:
eval_KL_model(V_train, V_test, [U1_train, U2_train], [U1_test, U2_test])

Fisher-Rao:    0.3938388560361861
Symmetric KL:  0.0240752766083486
L2 error:      0.25747135004291116
L1 error:      0.3722438869854835


In [12]:
eval_L2_model(V_train, V_test, [U1_train, U2_train], [U1_test, U2_test])

Fisher-Rao:    0.38970540860620667
Symmetric KL:  0.023850161075651598
L2 error:      0.2555180413915952
L1 error:      0.37014941154737036


In [13]:
eval_KL_model(V_train, V_test, [U1_train, U2_train], [U1_test, U2_test], center_data = False)

Fisher-Rao:    1.141862963697338
Symmetric KL:  0.20761975708178956
L2 error:      0.7489706273498107
L1 error:      1.0825563215510952


In [14]:
eval_L2_model(V_train, V_test, [U1_train, U2_train], [U1_test, U2_test], center_data = False)

Fisher-Rao:    1.1966443661220107
Symmetric KL:  0.23603413243385934
L2 error:      0.8081024870243325
L1 error:      1.157215326120599


# D17 Aitchison

In [15]:
# Dataset 17 Aitchison (with V[13] adjusted due to error in book)
V = np.array([
[0.27,0.28,0.45],
[0.02,0.03,0.95],
[0.12,0.16,0.72],
[0.83,0.02,0.15],
[0.24,0.22,0.54],
[0.16,0.20,0.64],
[0.31,0.08,0.61],
[0.05,0.85,0.10],
[0.06,0.06,0.88],
[0.08,0.31,0.61],
[0.18,0.20,0.62],
[0.17,0.19,0.64],
[0.04,0.17,0.79],
[0.08,0.25,0.67],
[0.11,0.34,0.55]])

U1 = np.array([
[0.70,0.07,0.23],
[0.19,0.16,0.65],
[0.18,0.26,0.54],
[0.02,0.02,0.96],
[0.08,0.16,0.76],
[0.14,0.18,0.68],
[0.16,0.11,0.73],
[0.04,0.06,0.90],
[0.06,0.54,0.40],
[0.12,0.22,0.66],
[0.06,0.02,0.92],
[0.16,0.04,0.80],
[0.27,0.17,0.56],
[0.21,0.51,0.28],
[0.15,0.15,0.70] ])

In [16]:
V_train, V_test = V[:12], V[12:]
U1_train, U1_test = U1[:12], U1[12:]

In [17]:
eval_KL_model(V_train, V_test, [U1_train], [U1_test])

Fisher-Rao:    1.243822297977255
Symmetric KL:  0.534509779806563
L2 error:      0.5575393699095036
L1 error:      0.8881891786261503


In [18]:
eval_L2_model(V_train, V_test, [U1_train], [U1_test])

Fisher-Rao:    1.1716720959817062
Symmetric KL:  0.4766113979085447
L2 error:      0.5141937028565229
L1 error:      0.7943933063157911


In [19]:
eval_KL_model(V_train, V_test, [U1_train], [U1_test], center_data = False)

Fisher-Rao:    2.087125906258782
Symmetric KL:  1.6253037903519312
L2 error:      1.1026764719736475
L1 error:      1.7690094548744115


In [20]:
eval_L2_model(V_train, V_test, [U1_train], [U1_test], center_data = False)

Fisher-Rao:    2.090729162208426
Symmetric KL:  1.6301387682323807
L2 error:      1.1049098924621183
L1 error:      1.7726035913451252


# Artificial  Dataset

Created by generating a dataset in $\mathbb{R}^{d-1}$ and then using the inverse ilr map to produce compositional data.

In [21]:
np.random.seed(200)

In [22]:
n, d, k = 30, 10, 4

In [23]:
U_vec = [ ilr_inv(np.random.randn(n,d-1)) for i in range(k)]

In [24]:
beta_true = [(-1)**j * 0.5 * k for j in range(1,k+1)]

In [25]:
V = ilr_inv(sum( beta_true[i] * ilr(U_vec[i]) for i in range(k)) + 0.2* np.random.randn(n,d-1) )

In [26]:
V_train, V_test = V[:20], V[20:]
U_train_vec = [U[:20] for U in U_vec]
U_test_vec = [U[20:] for U in U_vec]

In [27]:
eval_KL_model(V_train, V_test, U_train_vec, U_test_vec)

Fisher-Rao:    8.799269154289348
Symmetric KL:  11.838319859552247
L2 error:      3.610562371001399
L1 error:      5.976266269180193


In [28]:
eval_L2_model(V_train, V_test, U_train_vec, U_test_vec)

Fisher-Rao:    8.804243155122535
Symmetric KL:  11.802893034877115
L2 error:      3.6160491777625943
L1 error:      5.981286409423493


In [29]:
eval_KL_model(V_train, V_test, U_train_vec, U_test_vec, center_data = False)

Fisher-Rao:    0.7251876828898074
Symmetric KL:  0.06514220981605431
L2 error:      0.2840975782455637
L1 error:      0.48217691582155114


In [30]:
eval_L2_model(V_train, V_test, U_train_vec, U_test_vec, center_data = False)

Fisher-Rao:    0.8254282569572294
Symmetric KL:  0.08732074352649755
L2 error:      0.34703830889367143
L1 error:      0.5673855557828801
